In [3]:
import os

import torch
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import albumentations as A

from going_modular.dataloader.magface import create_magface_dataloader
from going_modular.model.MagFaceRecognition import MagFaceRecognition
from going_modular.train_eval.magface.train import fit
from going_modular.loss.MagLoss import MagLoss
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint
from going_modular.utils.transforms import RandomResizedCropRect, GaussianNoise

device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Đặt seed toàn cục
seed = 42
torch.manual_seed(seed)

CONFIGURATION = {
    # Thư mục
    'type': 'albedo',
    'train_dir': './Dataset/Albedo/train',
    'test_dir': './Dataset/Albedo/test',
    
    # Cấu hình train
    'backbone': 'iresnet18',
    'epochs': 500,
    'num_workers': 4,
    'batch_size': 16,
    'image_size': 224,
    'num_class': len(os.listdir('./Dataset/Albedo/train')),
    'embedding_size': 512,
    
    'learning_rate': 0.2,
    'weight_decay': 5e-4,
    'momentum': 0.9,
    'alpha': 0.9,
    
    # Hàm m(ai) giúp thay đổi ai từ 0.25 đến 1.6
    'scale': 64,
    'lambda_g': 20,
    'l_margin': 0.45, 
    'u_margin': 0.8,
    'l_a': 10, 
    'u_a': 110,
}

train_transform = A.Compose([
    RandomResizedCropRect(256),
    GaussianNoise(),
])

test_transform = A.Compose([
    A.Resize(height=CONFIGURATION['image_size'], width=CONFIGURATION['image_size'])
])

train_dataloader, test_dataloader = create_magface_dataloader(CONFIGURATION, train_transform, test_transform)

model = MagFaceRecognition(CONFIGURATION).to(device)
criterion = MagLoss(conf = CONFIGURATION)

optimizer = torch.optim.Adam(model.parameters(), lr=CONFIGURATION['learning_rate'])
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=40, T_mult=1, eta_min=1e-6)
checkpoint_path = os.path.abspath('checkpoint/magface/' + CONFIGURATION['type'] + '/models/checkpoint.pth')
modle_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1)
earlystop_dir = os.path.abspath('checkpoint/magface/' + CONFIGURATION['type'] + '/models')
early_stopping = MultiMetricEarlyStopping(
    monitor_keys=['test_cosine_auc', 'test_cosine_accuracy', 'test_euclidean_auc', 'test_euclidean_accuracy'],
    patience=40,
    mode='max',
    verbose=1,
    save_dir=earlystop_dir,
    start_from_epoch=30
)            

In [4]:
fit(CONFIGURATION, 0, model, device, train_dataloader, test_dataloader, criterion, optimizer, scheduler, early_stopping, modle_checkpoint)

Epoch 1:
	train: loss 3.721 | loss id   3.70 | top_1_acc 0.0067 | top_5_acc 0.0304 | cos_auc: 0.5849 | cos_acc: 0.2976 | eu_auc: 0.5715 | eu_acc: 0.6767
	test: cos_auc: 0.6139 | cos_acc: 0.7079 | eu_auc: 0.5953 | eu_acc: 0.6796
	Saving model and optimizer state to /media/vohoang/WorkSpace/ubuntu/projects/in-process/Bachelor-s-Project/checkpoint/magface/albedo/models/checkpoint.pth
Epoch 2:
	train: loss 2.607 | loss id   2.58 | top_1_acc 0.0045 | top_5_acc 0.0247 | cos_auc: 0.5990 | cos_acc: 0.4062 | eu_auc: 0.5930 | eu_acc: 0.5341
	test: cos_auc: 0.6525 | cos_acc: 0.5911 | eu_auc: 0.6343 | eu_acc: 0.6248
	Saving model and optimizer state to /media/vohoang/WorkSpace/ubuntu/projects/in-process/Bachelor-s-Project/checkpoint/magface/albedo/models/checkpoint.pth
Epoch 3:
	train: loss 0.617 | loss id   0.59 | top_1_acc 0.0079 | top_5_acc 0.0382 | cos_auc: 0.6247 | cos_acc: 0.4006 | eu_auc: 0.5983 | eu_acc: 0.4052
	test: cos_auc: 0.6964 | cos_acc: 0.5238 | eu_auc: 0.6519 | eu_acc: 0.6092
	Sav

In [ ]:
checkpoint = torch.load('checkpoint/magface/' + CONFIGURATION['type'] + '/models/checkpoint.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
print(epoch)

In [ ]:
fit(CONFIGURATION, 246, model, device, train_dataloader, test_dataloader, criterion, optimizer, scheduler, early_stopping, modle_checkpoint)